In [49]:
from google.cloud import storage
import os
from dotenv import load_dotenv
import json
import sys
from podcast_ad_skipper.Spectrogram_Conversion import create_spectrogram, spectrogram_to_numpy
import pandas as pd
from podcast_ad_skipper.upload_to_gcloud import auth_gc
import librosa

from google.auth.exceptions import GoogleAuthError
from google.cloud import storage
from google.oauth2 import service_account
from google.cloud import bigquery

from podcast_ad_skipper.params import *


In [50]:
storage_client = auth_gc()

Authenticated successfully! ✅


In [65]:
def retrieve_files_in_folder(bucket_name, prefixes):
    file_list = []
    bucket = storage_client.bucket(bucket_name)
    file_list_google_object = bucket.list_blobs(prefix=prefixes)
    for file in file_list_google_object:
        file_list.append(file)
    return file_list


In [68]:
file_list = retrieve_files_in_folder(bucket_name, prefixes)


In [69]:
type(file_list)

list

In [70]:
def open_gcs_file(file):
    audio_file = file.open('rb')
    return audio_file

In [71]:
open_file = open_gcs_file(file_list[0])

In [72]:
def create_spectrogram(open_file):
    #y is an array representing the frequency of the audio signal at each sample.
    #sr is the sampling rate (samples per second),
    y, sr = librosa.load(open_file)
    spectrogram = librosa.stft(y)

    #Transforms to decibel scale (logarithmic), which gives more emphasis to amplitude changes in high volumes
    spectrogram_db = librosa.amplitude_to_db(abs(spectrogram))
    return spectrogram_db

In [73]:
create_spectrogram(open_file)

array([[-10.198136, -15.113926, -29.613918, ..., -29.926647, -24.283981,
        -18.033857],
       [-10.190883, -19.369791, -25.763617, ..., -27.13015 , -23.894823,
        -19.20064 ],
       [ -9.43372 , -15.497618, -26.510141, ..., -30.378845, -29.13692 ,
        -23.978172],
       ...,
       [-38.53384 , -38.53384 , -38.53384 , ..., -38.53384 , -38.53384 ,
        -38.53384 ],
       [-38.53384 , -38.53384 , -38.53384 , ..., -38.53384 , -38.53384 ,
        -38.53384 ],
       [-38.53384 , -38.53384 , -38.53384 , ..., -38.53384 , -38.53384 ,
        -38.53384 ]], dtype=float32)

In [74]:
spectrogram_list=[]

In [83]:
#main.py
prefixes =GCP_PREFIXES[0]
bucket_name = BUCKET_NAME

def we_have_to_Change_the_name(bucket_name, prefixes):
    file_list = retrieve_files_in_folder(bucket_name, prefixes)
    for file in file_list[:6]:
        open_file = open_gcs_file(file)
        spectrogram_db = create_spectrogram(open_file)
        spectrogram_list.append(spectrogram_db)


In [82]:
we_have_to_Change_the_name(bucket_name, prefixes)

In [78]:
len(spectrogram_list)

6

In [84]:
spectrogram_list[0]

array([[-10.198136, -15.113926, -29.613918, ..., -29.926647, -24.283981,
        -18.033857],
       [-10.190883, -19.369791, -25.763617, ..., -27.13015 , -23.894823,
        -19.20064 ],
       [ -9.43372 , -15.497618, -26.510141, ..., -30.378845, -29.13692 ,
        -23.978172],
       ...,
       [-38.53384 , -38.53384 , -38.53384 , ..., -38.53384 , -38.53384 ,
        -38.53384 ],
       [-38.53384 , -38.53384 , -38.53384 , ..., -38.53384 , -38.53384 ,
        -38.53384 ],
       [-38.53384 , -38.53384 , -38.53384 , ..., -38.53384 , -38.53384 ,
        -38.53384 ]], dtype=float32)